An overview of the data cleaning steps implemented in lovelyrita.

In [8]:
from lovelyrita.clean import clean

In [9]:
clean??

Signature: clean(dataframe)
Source:   
def clean(dataframe):
    """Apply a series of data cleaning steps to a dataframe of raw data

    Parameters
    ----------
    dataframe : pandas.DataFrame

    Returns
    -------
    A cleaned DataFrame
    """

    drop_void(dataframe)

    replace(dataframe.street)

    addresses = parse_addresses(dataframe.street)
    dataframe.update(addresses)

    dataframe['ticket_issue_datetime'] = get_datetime(dataframe)
    dataframe['ticket_issue_datetime'] = impute_missing_times(dataframe.ticket_issue_datetime)
    dataframe.drop(['ticket_issue_time', 'ticket_issue_date'], axis=1, inplace=True)

    for column in find_dollar_columns(dataframe):
        dataframe[column] = convert_dollar_to_float(dataframe[column])

    return dataframe
File:      /projects/lovely-rita/lovelyrita/clean.py
Type:      function


Data cleaning involves several stages:

# Drop voided citations
Some citations are voided as indicated by the word "VOID" or "ZVOIDZ" in the address field. Others have no ticket number. Drop these rows from the dataframe.

In [3]:
from lovelyrita.clean import drop_void

In [4]:
drop_void??

Signature: drop_void(dataframe, inplace=True)
Source:   
def drop_void(dataframe, inplace=True):
    """Drop voided citations

    Parameters
    ----------
    dataframe : pandas.DataFrame
    inplace : bool

    Returns
    -------
    If `inplace` is False, return the dataframe with voided citations dropped
    """
    void_indices = dataframe.street.str.contains(r'^Z?VOIDZ?')
    null_indices = dataframe.ticket_number.isnull()
    drop_indices = np.logical_or(void_indices, null_indices).nonzero()[0]
    if inplace:
        dataframe.drop(drop_indices, inplace=True)
    else:
        return dataframe.drop(drop_indices, inplace=False)
File:      /projects/lovely-rita/lovelyrita/clean.py
Type:      function


# Parse addresses
We parse the address to produce a street number and street name. For example, "123 MAIN STREET" will produce "123" and "MAIN STREET". This can simplify geocoding queries later on, as certain APIs can take street name and number as separate fields.

Several address patterns are detected.

In [10]:
from lovelyrita.addresses import parse_addresses, REPLACEMENTS

In [15]:
parse_addresses??

Signature: parse_addresses(addresses)
Source:   
def parse_addresses(addresses):
    """Parse addresses into street name and number according to several rules.

    Parameter
    ---------
    addresses : pandas.Series

    Returns
    -------
    A DataFrame containing street name and street column for those rows that were successfully 
    parsed
    """
    # Many addresses are in parking lots. Those will not have street numbers, so we should treat them separately. We will only concern ourselves with potential street addresses.

    lot_indices = addresses.str.contains('^[A-Z]LOT.*LOT$')
    street_addresses = addresses.loc[~lot_indices]

    street = pd.DataFrame({'street_name': None, 'street_no': None},
                          index=addresses.index)

    new_street = parse_123_main_street(street_addresses)
    street.update(new_street)

    new_street = parse_P123_main_street(street_addresses)
    street.update(new_street)

    return street
File:      /projects/lovely-rita/love

# Replace text
Fix things by providing a pattern (as a [regular expression](https://regexr.com/)) and replacement text for that pattern. This is the most fine-tuned way to fix problems and should be used sparingly, since each replacement takes some time.

Here are the default patterns and replacements.

In [11]:
REPLACEMENTS

[('^ONE ', '1 '),
 ('^TWO ', '2 '),
 (' -', '-'),
 (' TERM$', ' TERMINAL'),
 ('^#', '')]

# Parse ticket issue times
Ticket issue times are provided as strings in the raw data. They can be parsed into `datetime` objects that support many useful operations, e.g., finding citations in a date range.

In [12]:
from lovelyrita.clean import get_datetime, impute_missing_times

In [13]:
get_datetime??

Signature: get_datetime(dataframe)
Source:   
def get_datetime(dataframe):
    """Get a datatime for each row in a DataFrame

    Parameters
    ----------
    dataframe : pandas.DataFrame
        A dataframe with `ticket_issue_date` and `ticket_issue_time` columns

    Returns
    -------
    A Series of datetime values
    """
    dt = dataframe['ticket_issue_date'] + ' ' + dataframe['ticket_issue_time']
    datetime_format = infer_datetime_format(dt)
    return pd.to_datetime(dt, format=datetime_format)
File:      /projects/lovely-rita/lovelyrita/clean.py
Type:      function


## Inferring the times for missing data

Once we have the times as `datetime` objects, we can guess the ticket issue times where they are missing by interpolating between known times. Caveat: this assumes the tickets are listed in chronological order.

In [14]:
impute_missing_times??

Signature: impute_missing_times(datetimes)
Source:   
def impute_missing_times(datetimes):
    """Fill in missing times by interpolating surrounding times

    Parameters
    ----------
    datetimes : pandas.Series

    Returns
    -------
    The original Series with missing times replaced by interpolated times
    """

    # get valid start and stop indices for null ranges
    n_rows = len(datetimes)

    null_indices = datetimes.isnull().nonzero()[0]

    # remove all but first in consecutive sequences of nulls
    valid_starts = null_indices[1:][np.diff(null_indices) > 1]
    if null_indices[0] > 0:
        valid_starts = np.r_[null_indices[0], valid_starts]
    valid_starts -= 1

    # remove all but final in consecutive sequences of nulls
    valid_ends = null_indices[:-1][np.diff(null_indices) > 1]
    if null_indices[-1] < (n_rows - 1):
        valid_ends = np.r_[valid_ends, null_indices[-1]]
    valid_ends += 1

    for valid_start, valid_end in zip(valid_starts, valid_ends):

# Convert dollar fields to numeric

In [15]:
from lovelyrita.clean import convert_dollar_to_float

In [16]:
convert_dollar_to_float??

Signature: convert_dollar_to_float(dollars)
Source:   
def convert_dollar_to_float(dollars):
    return dollars.replace('\$', '', regex=True).astype('float32')
File:      /projects/lovely-rita/lovelyrita/clean.py
Type:      function
